In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
%matplotlib inline

import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

In [4]:
train = pd.read_csv("data/train.csv")

In [5]:
y_label = train["label"]

In [6]:
train = train.drop("label", axis=1)

In [7]:
train = np.array(train).reshape(32, 32, 3, -1).transpose(3,0,1,2)

In [8]:
y_label[y_label == 10] = 0
y_train = np.zeros([len(y_label), 10])
y_train[np.arange(len(y_label)), y_label] = 1

In [14]:
tf.reset_default_graph()

In [12]:
network = input_data(shape=[None, 32, 32, 3])
network = conv_2d(network, 32, 3, activation='relu')
network = conv_2d(network, 32, 3, activation='relu')
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
#network = max_pool_2d(network, 2)
network = conv_2d(network, 128, 3, activation='relu')
network = conv_2d(network, 128, 3, activation='relu')
network = max_pool_2d(network, 2)
network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 10, activation='softmax')
regress_layer = regression(network, optimizer='adam', 
                     loss='categorical_crossentropy',
                     learning_rate=1e-3)

In [13]:
model = tflearn.DNN(regress_layer, checkpoint_path='model.tfl.ckpt', 
                    tensorboard_verbose=0, tensorboard_dir="tmp/logs")
model.fit(train, y_train, batch_size=128, n_epoch=20, validation_set=0.1, 
          shuffle=True, snapshot_step=10000, run_id="LeNet")

Training Step: 1419  | total loss: 0.29824
| Adam | epoch: 002 | loss: 0.29824 -- iter: 49536/65931


KeyboardInterrupt: 

In [ ]:
model.save(time.strftime(time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())))

In [ ]:
x_test = pd.read_csv("data/test.csv")

In [ ]:
x_test = np.array(x_test).reshape(32, 32, 3, -1).transpose(3,0,1,2)

In [ ]:
plt.imshow(x_test[3].astype("uint8"))

In [ ]:
res_list = list()
for i in x_test:
    pred = model.predict(i.reshape(-1,32,32,3))
    res_list.append(np.argmax(pred))

In [ ]:
res_list = np.array(res_list)
res_list[res_list == 0] = 10

res_output = pd.Dataframe({
    "ImageId": range(len(pred))
    "label": pred
})
filename = "res_" + time.strftime(time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())) + ".csv"
res_output.to_csv(filename, index=False)